# Baseline on-the-fly 3DGS (original bootstrap)

This notebook runs the original on-the-fly pipeline with its geometric bootstrap and serves as the baseline for comparisons.

- Ensure your scene lives under `on-the-fly-nvs/data/my_scene/images`.
- Training outputs go to `on-the-fly-nvs/results/scene`.
- Use the PSNR notebooks to evaluate the held-out test views.

In [1]:
#@title 1) Cloner et installer on-the-fly-nvs + dépendances
!git clone --recursive https://github.com/graphdeco-inria/on-the-fly-nvs.git
%cd on-the-fly-nvs

!pip install -q -r requirements.txt

import torch
print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
assert device=="cuda", "GPU requis pour un entraînement fluide."


Cloning into 'on-the-fly-nvs'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 808 (delta 22), reused 14 (delta 13), pack-reused 773 (from 1)
Receiving objects: 100% (808/808), 5.20 MiB | 18.48 MiB/s, done.
Resolving deltas: 100% (259/259), done.
Submodule 'submodules/Depth-Anything-V2' (https://github.com/DepthAnything/Depth-Anything-V2.git) registered for path 'submodules/Depth-Anything-V2'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim) registered for path 'submodules/fused-ssim'
Submodule 'submodules/graphdecoviewer' (https://github.com/graphdeco-inria/graphdecoviewer.git) registered for path 'submodules/graphdecoviewer'
Cloning into '/content/on-the-fly-nvs/submodules/Depth-Anything-V2'...
remote: Enumerating objects: 142, done.        
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142 (from 1)        
Receiving objects: 100% (142/142),

In [ ]:
# Colab upload/download shims
try:
    from google.colab import files  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False
    class _FilesShim:
        def upload(self):
            print("Local mode: place files in the expected folders; no upload dialog.")
        def download(self, *args, **kwargs):
            print("Local mode: file saved on disk; no download dialog.")
    files = _FilesShim()

In [ ]:
# Sanity check
all_imgs = sorted([f for f in os.listdir(IMAGES_DIR) if f.lower().endswith((".jpg",".jpeg",".png"))])
print(f"Nombre d'images détectées: {len(all_imgs)}")
print("Exemples:", all_imgs[:3], "…", all_imgs[-3:])
assert len(all_imgs) >= 20, "Il faut au moins quelques images consécutives pour le bootstrap."

In [ ]:
#@title 3) Launch incremental training (repo default bootstrap)
import subprocess, shlex, os, textwrap, sys

# Let the repo do its internal bootstrap (~8 frames).
# Use --viewer_mode none for Colab, and a reasonable downsampling if 1500+ images.
env = os.environ.copy()
prev_pp = env.get("PYTHONPATH", "")
extra = os.getcwd()
env["PYTHONPATH"] = os.pathsep.join([p for p in [prev_pp, extra] if p])

cmd = f"python train.py -s {SOURCE_ROOT} -m {MODEL_DIR} --viewer_mode none --downsampling 2.5 --save_every 200 --display_runtimes"
print("\n--bootstrap_size not supported? → RUN (fallback):", cmd)
ret = subprocess.call(shlex.split(cmd), env=env)

print("Exit code:", ret)
assert ret == 0, "Training interrupted. Check logs above."


RUN (try with --bootstrap_size): python train.py -s data/my_scene -m results/my_scene --viewer_mode none --downsampling 2.5 --save_every 200 --display_runtimes --bootstrap_size 8

--bootstrap_size non supporté ?  → RUN (fallback): python train.py -s data/my_scene -m results/my_scene --viewer_mode none --downsampling 2.5 --save_every 200 --display_runtimes
Exit code: 0


In [ ]:
#@title 4) Render optimized COLMAP path (video + images)
import os, glob, subprocess, shlex

COLMAP_DIR = os.path.join(MODEL_DIR, "colmap")
assert os.path.isdir(COLMAP_DIR), "No 'colmap/' found (training incomplete?)."

OUT_RENDER = os.path.join(MODEL_DIR, "renders")
os.makedirs(OUT_RENDER, exist_ok=True)

cmd = f"python scripts/render_path.py -m {MODEL_DIR} --render_path {COLMAP_DIR} --out_dir {OUT_RENDER}"
print("RUN:", cmd)
ret = subprocess.call(shlex.split(cmd))
print("Exit code:", ret)

# Show produced artifacts
vid = os.path.join(OUT_RENDER, "rendered_path.mp4")
imgs = sorted(glob.glob(os.path.join(OUT_RENDER, "*.png")) + glob.glob(os.path.join(OUT_RENDER, "*.jpg")))
print("Video:", "OK" if os.path.exists(vid) else "missing")

RUN: python scripts/render_path.py -m results/my_scene --render_path results/my_scene/colmap --out_dir results/my_scene/renders
Exit code: 0
Vidéo : OK
Nb d'images rendues : 0
